# Url Loader Testing

In [10]:
from PyPDF2 import PdfReader
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [12]:
# location of the pdf file/files. 
reader = PdfReader('./pdfs/Sports-And-Exercise-Nutrition.pdf')

In [13]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [14]:
raw_text[:100]


'Lanham_bindex.indd   388Lanham_bindex.indd   388 7/28/2011   4:40:16 PM7/28/2011   4:40:16 PMSport a'

In [15]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

Created a chunk of size 1398, which is longer than the specified 1000
Created a chunk of size 1433, which is longer than the specified 1000
Created a chunk of size 1419, which is longer than the specified 1000
Created a chunk of size 1102, which is longer than the specified 1000
Created a chunk of size 1230, which is longer than the specified 1000
Created a chunk of size 1248, which is longer than the specified 1000
Created a chunk of size 1496, which is longer than the specified 1000
Created a chunk of size 1050, which is longer than the specified 1000
Created a chunk of size 2026, which is longer than the specified 1000
Created a chunk of size 1118, which is longer than the specified 1000
Created a chunk of size 1029, which is longer than the specified 1000
Created a chunk of size 1571, which is longer than the specified 1000
Created a chunk of size 1631, which is longer than the specified 1000
Created a chunk of size 2110, which is longer than the specified 1000
Created a chunk of s

In [16]:
len(texts)

2046

In [17]:
texts[0]

'Lanham_bindex.indd   388Lanham_bindex.indd   388 7/28/2011   4:40:16 PM7/28/2011   4:40:16 PMSport and Exercise Nutrition\nLanham_ffirs.indd   iLanham_ffirs.indd   i 7/28/2011   5:49:59 PM7/28/2011   5:49:59 PMIntroduction to Human Nutrition\nIntroduction to human nutrition: a global \nperspective on food and nutrition\nBody compositionEnergy metabolismNutrition and metabolism of proteins and amino acidsDigestion and metabolism of carbohydratesNutrition and metabolism of lipidsDietary reference standardsThe vitaminsMinerals and trace elementsMeasuring food intakeFood compositionFood and nutrition: policy and regulatory issuesNutrition research methodologyFood safety: a public health issue of growing importanceFood and nutrition-related diseases: the global challenge\nPublic Health Nutrition\nAn overview of public health nutritionNutrition epidemiologyFood choiceAssessment of nutritional status at individual and \n population level'

In [ ]:
# Download embeddings from OpenAI
embeddings = CohereEmbeddings()

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
docsearch

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = "who are the authors of the article?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)